In [1]:
import yaml
from models import *
import os

## Configuration



In [2]:
storage_class_rwo = "standard"
storage_class_rwx = "standard"

workspace_volume_size = "50Gi"
calrissian_volume_size = "50Gi"


## Volumes

Create the Volumes

### Workspace Volume

The workspace volume is persisted.

In [3]:
workspace_volume = Volume(
    name="workspace-volume",
    size=workspace_volume_size,
    claim_name="workspace-claim",
    mount_path="/workspace",
    storage_class=storage_class_rwo,
    access_modes=["ReadWriteOnce"],
    volume_mount=VolumeMount(name="workspace-volume", mount_path="/workspace"),
    persist=True,
)

### Calrissian Volume

This is a RWX volume, not persisted

In [4]:
calrissian_volume = Volume(
    name="calrissian-volume",
    claim_name="calrissian-claim",
    size=calrissian_volume_size,
    storage_class=storage_class_rwx,
    access_modes=["ReadWriteMany"],
    volume_mount=VolumeMount(name="calrissian-volume", mount_path="/calrissian"),
    persist=False,
)

## ConfigMaps

These configmaps are mounted as files on the pod.

### bash login

This file is used for the JupyterLab Terminal configuration

In [5]:
with open("config-maps/bash-login", "r") as f:
    content = f.read()

bash_login_cm = ConfigMap(
    name="bash-login",
    key="bash-login",
    content=content,
    readonly=True,
    persist=True,
    mount_path="/workspace/.bash_login",
)

#### bash.rc


In [6]:
with open("config-maps/bash-rc", "r") as f:
    content = f.read()
bash_rc_cm = ConfigMap(
    name="bash-rc",
    key="bash-rc",
    content=content,
    readonly=True,
    persist=True,
    mount_path="/workspace/.bashrc",
)

## Profiles

In [7]:
profiles = []

### Coder

In [8]:
coders = {
    "coder1": {
        "display_name": "Code Server Small",
        "slug": "eoepca_coder_slug_s",
        "cpu_limit": 2,
        "mem_limit": "8G",
    },
    "coder2": {
        "display_name": "Code Server Medium",
        "slug": "eoepca_coder_slug_m",
        "cpu_limit": 4,
        "mem_limit": "12G",
    },
}

for key, value in coders.items():
    coder_definition = ProfileDefinition(
        display_name=value["display_name"],
        slug=value["slug"],
        default=False,
        kubespawner_override=KubespawnerOverride(
            cpu_limit=value["cpu_limit"],
            mem_limit=value["mem_limit"],
            image="eoepca/pde-code-server:develop",
        ),
    )

    coder_profile = Profile(
        id=f"profile_studio_{key}",
        groups=["group-a", "group-b"],
        definition=coder_definition,
        node_selector={},
        volumes=[calrissian_volume, workspace_volume],
        config_maps=[
            bash_rc_cm,
        ],
        pod_env_vars={
            "HOME": "/workspace",
            "CONDA_ENVS_PATH": " /workspace/.envs",
        },
    )

    profiles.append(coder_profile)

## init.sh script

In [9]:
with open("./config-maps/init.sh", "r") as f:
    content = f.read()

init_cm = ConfigMap(
    name="init",
    key="init",
    content=content,
    readonly=True,
    persist=False,
    mount_path="/opt/init/.init.sh",
    default_mode="0660",
)


init_context_volume_mount = InitContainerVolumeMount(
    mount_path="/opt/init/.init.sh", name="init", sub_path="init"
)
init_container = InitContainer(
    name="init-file-on-volume",
    image="eoepca/pde-code-server:develop",
    command=["sh", "-c", "sh /opt/init/.init.sh"],
    volume_mounts=[
        VolumeMount(name="workspace-volume", mount_path="/workspace"),
        init_context_volume_mount,
    ],
)

eoepca_demo_init_script_profile = Profile(
    id=f"profile_demo_init_script",
    groups=["group-a", "group-b"],
    definition=ProfileDefinition(
        display_name="Coder demo init script",
        description="This profile is used to demonstrate the use of an init script",
        slug="eoepca_demo_init_script",
        default=False,
        kubespawner_override=KubespawnerOverride(
            cpu_guarantee=1,
            cpu_limit=2,
            mem_guarantee="4G",
            mem_limit="6G",
            image="eoepca/pde-code-server:develop",
        ),
    ),
    node_selector={},
    volumes=[calrissian_volume, workspace_volume],
    config_maps=[init_cm],
    pod_env_vars={
        "HOME": "/workspace",
        "CONDA_ENVS_PATH": "/workspace/.envs",
        "CONDARC": "/workspace/.condarc",
        "XDG_RUNTIME_DIR": "/workspace/.local",
        "CODE_SERVER_WS": "/workspace/mastering-app-package",
    },
    init_containers=[init_container],
)

In [10]:
profiles.append(eoepca_demo_init_script_profile)

eoepca_demo_init_script_profile = Profile(
    id=f"profile_demo_init_script",
    groups=["group-a", "group-b"],
    definition=ProfileDefinition(
        display_name="Coder demo init script",
        description="This profile is used to demonstrate the use of an init script",
        slug="eoepca_demo_init_script",
        default=False,
        kubespawner_override=KubespawnerOverride(
            cpu_guarantee=1,
            cpu_limit=2,
            mem_guarantee="4G",
            mem_limit="6G",
            image="eoepca/pde-code-server:develop",
        ),
    ),
    node_selector={},
    volumes=[calrissian_volume, workspace_volume],
    config_maps=[init_cm],
    pod_env_vars={
        "HOME": "/workspace",
        "CONDA_ENVS_PATH": "/workspace/.envs",
        "CONDARC": "/workspace/.condarc",
        "XDG_RUNTIME_DIR": "/workspace/.local",
        "CODE_SERVER_WS": "/workspace/mastering-app-package",
    },
    init_containers=[init_container],
)

## JupyterLab

In [11]:
image = "jupyter/scipy-notebook"


eoepca_jupyter_lab_profile = Profile(
    id="profile_jupyter_lab",
    groups=["group-c"],
    definition=ProfileDefinition(
        display_name="Jupyter Lab",
        description="Jupyter Lab with Python 3.11",
        slug="eoepca_jupyter_lab",
        default=False,
        kubespawner_override=KubespawnerOverride(
            cpu_guarantee=1,
            cpu_limit=2,
            mem_guarantee="4G",
            mem_limit="6G",
            image=image,
        ),
    ),
    node_selector={},
    volumes=[workspace_volume],
    config_maps=[],
    pod_env_vars={
        "HOME": "/workspace",
        "XDG_RUNTIME_DIR": "/workspace/.local",
        "XDG_CONFIG_HOME": "/workspace/.config",
    },
)

profiles.append(eoepca_jupyter_lab_profile)

## Image pull secret


In [12]:
image_pull_secret = ImagePullSecret(
    name="cr-config",
    persist=False,
    data="",
    data="",
)

In [13]:
image = "jupyter/scipy-notebook"


eoepca_jupyter_lab_profile_2 = Profile(
    id="profile_jupyter_lab_2",
    groups=["group-c"],
    definition=ProfileDefinition(
        display_name="Jupyter Lab - profile 2",
        description="Jupyter Lab with Python 3.11 - demoes the use of an image pull secret",
        slug="eoepca_jupyter_lab_2",
        default=False,
        kubespawner_override=KubespawnerOverride(
            cpu_guarantee=1,
            cpu_limit=2,
            mem_guarantee="4G",
            mem_limit="6G",
            image=image,
        ),
    ),
    node_selector={},
    volumes=[workspace_volume],
    config_maps=[],
    pod_env_vars={
        "HOME": "/workspace",
        "XDG_RUNTIME_DIR": "/workspace/.local",
        "XDG_CONFIG_HOME": "/workspace/.config",
    },
    image_pull_secrets=[image_pull_secret],
)

profiles.append(eoepca_jupyter_lab_profile_2)

## Configuration

In [14]:
config = Config(
    profiles=profiles
)

with open(
    "../files/hub/config.yml", "w"
) as file:
    yaml.dump(config.dict(), file)

In [1]:
# profiles

[Profile(id='profile_studio_coder1', groups=['group-a', 'group-b'], definition=ProfileDefinition(display_name='Code Server Small', description=None, slug='ellip_studio_coder_slug_s', default=False, kubespawner_override=KubespawnerOverride(cpu_limit=2, cpu_guarantee=None, mem_limit='8G', mem_guarantee=None, image='eoepca/pde-code-server:develop', extra_resource_limits={}, extra_resource_guarantees={})), config_maps=[ConfigMap(name='bash-rc', key='bash-rc', mount_path='/workspace/.bashrc', default_mode=None, readonly=True, content='alias ll="ls -l"\nalias calrissian="/opt/conda/bin/calrissian --pod-nodeselectors /etc/calrissian/pod-node-selector.yml --stdout /calrissian/results.json --max-ram 16G --max-cores "8" --tmp-outdir-prefix /calrissian/tmp/ --outdir /calrissian/"\nalias cwltool="/opt/conda/bin/cwltool --podman"\n. /home/jovyan/.bashrc\n# >>> conda initialize >>>\n# !! Contents within this block are managed by \'conda init\' !!\n__conda_setup="$(\'/opt/conda/bin/conda\' \'shell.ba